In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
from utils import generate_table, Cell, to_latex, merge_cols, simple_to_latex

In [3]:
data = pd.read_csv("../../results/adult/balanced-eval.csv")

In [4]:
data["Group"].value_counts()

ranking-fdm.old.on_recon.balanced_eval.miss_s         30
perfect-cluster.old.on_recon.balanced_eval.partial    30
perfect-cluster.old.on_recon.balanced_eval.miss_s     30
kmeans-fdm.old.on_recon.balanced_eval.miss_s          30
no-cluster-fdm.old.on_recon.balanced_eval.partial     30
ranking-fdm.old.on_recon.balanced_eval.partial        30
no-cluster-fdm.old.on_recon.balanced_eval.miss_s      30
kmeans-fdm.old.on_recon.balanced_eval.partial         30
Name: Group, dtype: int64

In [5]:
data.columns

Index(['Name', 'Created', 'Runtime', 'Group', 'bias.log_dataset',
       'bias.missing_s', 'clust.method', 'fdm.balanced_context',
       'misc.log_method', 'Accuracy (LRCV)',
       'Accuracy (Logistic Regression (C=1.0))',
       'Accuracy (pytorch_classifier)', 'Accuracy Discriminator (zy)',
       'Accuracy_sex_Male_0.0 (LRCV)',
       'Accuracy_sex_Male_0.0 (Logistic Regression (C=1.0))',
       'Accuracy_sex_Male_0.0-sex_Male_1.0 (LRCV)',
       'Accuracy_sex_Male_0.0-sex_Male_1.0 (Logistic Regression (C=1.0))',
       'Accuracy_sex_Male_0.0-sex_Male_1.0 (pytorch_classifier)',
       'Accuracy_sex_Male_0.0÷sex_Male_1.0 (LRCV)',
       'Accuracy_sex_Male_0.0÷sex_Male_1.0 (Logistic Regression (C=1.0))',
       'Accuracy_sex_Male_0.0÷sex_Male_1.0 (pytorch_classifier)',
       'Accuracy_sex_Male_1.0 (LRCV)',
       'Accuracy_sex_Male_1.0 (Logistic Regression (C=1.0))',
       'Accuracy_sex_Male_1.0 (pytorch_classifier)', 'Clust/Context ARI',
       'Clust/Context Acc y=0 s=0', 'Clust

In [6]:
data["misc.log_method"].value_counts()

kmeans-fdm         60
perfect-cluster    60
no-cluster-fdm     60
ranking-fdm        60
Name: misc.log_method, dtype: int64

In [7]:
data = data.replace({"misc.log_method": {
    "kmeans-fdm": r"\texttt{ZSF+bal.\ (k-means)}",
    "ranking-fdm": r"\texttt{ZSF+bal.\ (ranking)}",
    "no-cluster-fdm": r"\texttt{ZSF}",
    "perfect-cluster": r"\texttt{ZSF+bal.\ (ground truth)}",
}}, inplace=False)

In [8]:
def table(data, filter_, cols):
    return (
        data
        .query(filter_)
        .groupby("Group")
        .agg(Cell(round_to=3))[list(cols)].rename(columns=cols, inplace=False)
    )

In [9]:
cols = {
    'Clust/Context Accuracy': "Clust Acc",
#     'Clust/Context NMI': "Clust NMI",
#     'Clust/Context ARI': "Clust ARI",
    "Accuracy (Logistic Regression (C=1.0))": "Accuracy",
    'prob_pos_sex_Male_0.0÷sex_Male_1.0 (Logistic Regression (C=1.0))': "AR ratio",
#     "Renyi preds and s (Logistic Regression (C=1.0))": "Renyi corr",
    "TPR_sex_Male_0.0÷sex_Male_1.0 (Logistic Regression (C=1.0))": "TPR ratio",
    'TNR_sex_Male_0.0÷sex_Male_1.0 (Logistic Regression (C=1.0))': "TNR ratio",
}

In [10]:
bias = "partial"

In [11]:
res = (data.query(f"`bias.log_dataset`.str.contains('{bias}')")[["misc.log_method"] + list(cols)]
        .groupby(["misc.log_method"], sort=False)
        .agg(Cell(round_to=3)).rename(columns=cols, inplace=False)
        .reset_index(level=["misc.log_method"], inplace=False)
        .rename(columns={"misc.log_method": "Method"}, inplace=False)
       )
res

,Method,Clust Acc,Accuracy,AR ratio,TPR ratio,TNR ratio
0,\texttt{ZSF},N/A,0.691 $\pm$ 0.014,0.313 $\pm$ 0.066,0.347 $\pm$ 0.07,0.859 $\pm$ 0.029
1,\texttt{ZSF+bal.\ (k-means)},0.388 $\pm$ 0.038,0.682 $\pm$ 0.027,0.262 $\pm$ 0.093,0.301 $\pm$ 0.109,0.835 $\pm$ 0.042
2,\texttt{ZSF+bal.\ (ranking)},0.699 $\pm$ 0.003,0.708 $\pm$ 0.017,0.488 $\pm$ 0.078,0.551 $\pm$ 0.085,0.909 $\pm$ 0.027
3,\texttt{ZSF+bal.\ (ground truth)},N/A,0.731 $\pm$ 0.011,0.556 $\pm$ 0.065,0.63 $\pm$ 0.072,0.897 $\pm$ 0.023


In [12]:
# baseline files
missing_demo = {
    r"Kamiran \& Calders (2012) LR": "adult_Kamiran & Calders LR_lr_baselines.csv",
    "LRCV": "adult_LRCV_lr_baselines.csv",
    "LR": "adult_Logistic Regression (C=1.0)_lr_baselines.csv",
    "Majority": "adult_Majority_lr_baselines.csv",
    "SVM (lin)": "adult_SVM (linear)_lr_baselines.csv",
    "SVM": "adult_SVM_lr_baselines.csv",
    r"Kamiran \& Calders (2012) MLP": "adult_cnn_baseline_60epochs.csv",
    "DRO": "adult_dro_baseline_60epochs.csv",
}
partial_outcome = {
    r"Kamiran \& Calders (2012) LR": "adult_Kamiran & Calders LR_lr_baselines.csv",
    "LRCV": "adult_LRCV_lr_baselines.csv",
    "LR": "adult_Logistic Regression (C=1.0)_lr_baselines.csv",
    "Majority": "adult_Majority_lr_baselines.csv",
    "SVM (lin)": "adult_SVM (linear)_lr_baselines.csv",
    "SVM": "adult_SVM_lr_baselines.csv",
    r"Kamiran \& Calders (2012) MLP": "adult_cnn_baseline_60epochs.csv",
    "DRO": "adult_dro_baseline_60epochs.csv",
}

In [13]:
def collate(file_dict, exp_group, dir_):
    df_all = pd.DataFrame()
    for log_method, filename in file_dict.items():
        df = pd.read_csv(Path("../../results/adult") / dir_ / filename)
        df.insert(0, "exp_group", exp_group)
        df.insert(0, "log_method", log_method)
        if log_method == "DRO":
            df["log_method"] = r"\texttt{FWD \cite{HasSriNamLia18}}" + " (" + df["eta"].astype(str) + ")"
        df_all = pd.concat([df_all, df], axis="index", ignore_index=True, sort=False)
    return df_all

In [14]:
baselines = pd.concat([collate(missing_demo, "miss_s", "missing_demo"), collate(partial_outcome, "partial", "partial_outcome")])

In [15]:
merge_cols(baselines, 'prob_pos_sex_Male_0.0÷sex_Male_1.0', 'prob_pos_sens_Label_0.0÷sens_Label_1.0')
merge_cols(baselines, 'TPR_sex_Male_0.0÷sex_Male_1.0', 'TPR_sens_Label_0.0÷sens_Label_1.0')
merge_cols(baselines, 'TNR_sex_Male_0.0÷sex_Male_1.0', 'TNR_sens_Label_0.0÷sens_Label_1.0')

In [16]:
bl_cols = {
 'Accuracy': "Accuracy",
#  'Renyi preds and s': "Renyi",
 'prob_pos_sex_Male_0.0÷sex_Male_1.0': "AR ratio",
#  "Renyi preds and s": "Renyi corr",
 'TPR_sex_Male_0.0÷sex_Male_1.0': "TPR ratio",
 'TNR_sex_Male_0.0÷sex_Male_1.0': "TNR ratio",
}

In [17]:
res2 = (baselines.query(f"exp_group == '{bias}'")[["log_method"] + list(bl_cols)]
        .groupby(["log_method"], sort=False)
        .agg(Cell(round_to=3)).rename(columns=bl_cols, inplace=False)
        .reset_index(level=["log_method"], inplace=False)
        .rename(columns={"log_method": "Method"}, inplace=False)
       )
res2

,Method,Accuracy,AR ratio,TPR ratio,TNR ratio
0,Kamiran \& Calders (2012) LR,0.64 $\pm$ 0.011,0.234 $\pm$ 0.034,0.259 $\pm$ 0.038,0.946 $\pm$ 0.014
1,LRCV,0.662 $\pm$ 0.009,0.182 $\pm$ 0.031,0.208 $\pm$ 0.036,0.906 $\pm$ 0.018
2,LR,0.664 $\pm$ 0.009,0.192 $\pm$ 0.026,0.219 $\pm$ 0.031,0.906 $\pm$ 0.019
3,Majority,0.5 $\pm$ 0.0,N/A,N/A,1.0 $\pm$ 0.0
4,SVM (lin),0.664 $\pm$ 0.009,0.198 $\pm$ 0.027,0.224 $\pm$ 0.032,0.911 $\pm$ 0.017
5,SVM,0.664 $\pm$ 0.01,0.242 $\pm$ 0.035,0.276 $\pm$ 0.041,0.916 $\pm$ 0.012
6,Kamiran \& Calders (2012) MLP,0.681 $\pm$ 0.009,0.151 $\pm$ 0.023,0.182 $\pm$ 0.026,0.79 $\pm$ 0.033
7,\texttt{FWD \cite{HasSriNamLia18}} (0.1),0.681 $\pm$ 0.008,0.15 $\pm$ 0.022,0.182 $\pm$ 0.024,0.79 $\pm$ 0.035
8,\texttt{FWD \cite{HasSriNamLia18}} (1.0),0.5 $\pm$ 0.0,1.0 $\pm$ 0.0,1.0 $\pm$ 0.0,1.0 $\pm$ 0.0
9,\texttt{FWD \cite{HasSriNamLia18}} (0.3),0.681 $\pm$ 0.009,0.149 $\pm$ 0.025,0.18 $\pm$ 0.028,0.79 $\pm$ 0.034


In [18]:
table = pd.concat([res, res2], sort=False).reset_index(drop=True)
table

,Method,Clust Acc,Accuracy,AR ratio,TPR ratio,TNR ratio
0,\texttt{ZSF},N/A,0.691 $\pm$ 0.014,0.313 $\pm$ 0.066,0.347 $\pm$ 0.07,0.859 $\pm$ 0.029
1,\texttt{ZSF+bal.\ (k-means)},0.388 $\pm$ 0.038,0.682 $\pm$ 0.027,0.262 $\pm$ 0.093,0.301 $\pm$ 0.109,0.835 $\pm$ 0.042
2,\texttt{ZSF+bal.\ (ranking)},0.699 $\pm$ 0.003,0.708 $\pm$ 0.017,0.488 $\pm$ 0.078,0.551 $\pm$ 0.085,0.909 $\pm$ 0.027
3,\texttt{ZSF+bal.\ (ground truth)},N/A,0.731 $\pm$ 0.011,0.556 $\pm$ 0.065,0.63 $\pm$ 0.072,0.897 $\pm$ 0.023
4,Kamiran \& Calders (2012) LR,NaN,0.64 $\pm$ 0.011,0.234 $\pm$ 0.034,0.259 $\pm$ 0.038,0.946 $\pm$ 0.014
5,LRCV,NaN,0.662 $\pm$ 0.009,0.182 $\pm$ 0.031,0.208 $\pm$ 0.036,0.906 $\pm$ 0.018
6,LR,NaN,0.664 $\pm$ 0.009,0.192 $\pm$ 0.026,0.219 $\pm$ 0.031,0.906 $\pm$ 0.019
7,Majority,NaN,0.5 $\pm$ 0.0,N/A,N/A,1.0 $\pm$ 0.0
8,SVM (lin),NaN,0.664 $\pm$ 0.009,0.198 $\pm$ 0.027,0.224 $\pm$ 0.032,0.911 $\pm$ 0.017
9,SVM,NaN,0.664 $\pm$ 0.01,0.242 $\pm$ 0.035,0.276 $\pm$ 0.041,0.916 $\pm$ 0.012


In [19]:
simple_to_latex(table)

\begin{tabular}{llllll}
\toprule
                                    Method &          Clust Acc &           Accuracy &           AR ratio &          TPR ratio &          TNR ratio \\
\midrule
                              \texttt{ZSF} &                N/A &  0.691 $\pm$ 0.014 &  0.313 $\pm$ 0.066 &   0.347 $\pm$ 0.07 &  0.859 $\pm$ 0.029 \\
              \texttt{ZSF+bal.\ (k-means)} &  0.388 $\pm$ 0.038 &  0.682 $\pm$ 0.027 &  0.262 $\pm$ 0.093 &  0.301 $\pm$ 0.109 &  0.835 $\pm$ 0.042 \\
              \texttt{ZSF+bal.\ (ranking)} &  0.699 $\pm$ 0.003 &  0.708 $\pm$ 0.017 &  0.488 $\pm$ 0.078 &  0.551 $\pm$ 0.085 &  0.909 $\pm$ 0.027 \\
         \texttt{ZSF+bal.\ (ground truth)} &                N/A &  0.731 $\pm$ 0.011 &  0.556 $\pm$ 0.065 &   0.63 $\pm$ 0.072 &  0.897 $\pm$ 0.023 \\
              Kamiran \& Calders (2012) LR &                NaN &   0.64 $\pm$ 0.011 &  0.234 $\pm$ 0.034 &  0.259 $\pm$ 0.038 &  0.946 $\pm$ 0.014 \\
                                      LRCV &        